In [ ]:
# Install dependencies

In [1]:
!pip install openai pyyaml

In [ ]:
# Imports and setup

In [5]:
import yaml
import xml.etree.ElementTree as ET
from openai import OpenAI
import os
import getpass
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
# Load platform.yaml

In [6]:
def load_platform_spec(path):
    with open(path, 'r') as f:
        return yaml.safe_load(f)

In [ ]:
# Parse BPMN and handle tasks + subprocesses (Custom parser to avoid NetworkX compatibility issue)

In [7]:
def parse_bpmn_tasks(file_path):
    with open(file_path, 'r') as f:
        xml_string = f.read()
    xml_string = xml_string.replace(' & ', ' &amp; ')  # Escape & in names
    root = ET.fromstring(xml_string)
    namespaces = {'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'}
    tasks = []
    for process in root.findall('.//bpmn:process', namespaces):
        for task_elem in process.iter():
            if task_elem.tag in ['{http://www.omg.org/spec/BPMN/20100524/MODEL}task', '{http://www.omg.org/spec/BPMN/20100524/MODEL}sendTask', '{http://www.omg.org/spec/BPMN/20100524/MODEL}receiveTask', '{http://www.omg.org/spec/BPMN/20100524/MODEL}userTask']:
                task_type = task_elem.tag.split('}')[-1]
                task_info = {
                    'id': task_elem.get('id'),
                    'name': task_elem.get('name', 'Unnamed'),
                    'type': task_type
                }
                tasks.append(task_info)
        for sub in process.findall('.//bpmn:subProcess', namespaces):
            task_info = {
                'id': sub.get('id'),
                'name': sub.get('name', 'Unnamed'),
                'type': 'subProcess',
                'subtype': 'embedded'
            }
            tasks.append(task_info)
        for call in process.findall('.//bpmn:callActivity', namespaces):
            task_info = {
                'id': call.get('id'),
                'name': call.get('name', 'Unnamed'),
                'type': 'callActivity',
                'subtype': 'call-activity',
                'calledElement': call.get('calledElement', 'Unknown')
            }
            tasks.append(task_info)
    return tasks

In [ ]:
# Governance-aware LLM code generator

In [8]:
def generate_task_code(task, platform_spec, model="gpt-4"):
    devops = platform_spec.get("devops_governance", {})
    security = platform_spec.get("security_governance", {})
    llm_settings = platform_spec.get("llm_guidance", {})

    prompt = f"""
    You are a platform-aware code generator.

    Generate {platform_spec['language']} code for a BPMN task.
    Task Name: \"{task['name']}\"
    Task Type: {task['type']}
    Platform Stack: {platform_spec['framework']}
    Architecture: {', '.join(platform_spec.get('architecture_patterns', []))}

    DevOps Governance:
      - Source: {devops.get('source', 'N/A')}
      - Enforced By: {devops.get('enforced_by', 'N/A')}
      - DORA Tracking: {devops.get('dora_metrics', {}).get('source_of_truth', 'N/A')}
      - Auto-inject Metrics: {llm_settings.get('auto_inject_metrics', True)}

    Security Governance:
      - Source: {security.get('source', 'N/A')}
      - Auth: {security.get('authentication', {}).get('protocol', 'N/A')}
      - API Gateway: {security.get('api_gateway', {}).get('provider', 'N/A')}

    Instructions:
      - Do NOT auto-inject Prometheus or OpenTelemetry metrics unless explicitly required.
      - Respect all DevOps and Security governance references.
      - Code should be governance-compliant, not ad-hoc.
    """

    if task.get("subtype") == "call-activity":
        prompt += f"""
        This is a CallActivity subprocess. Generate code that calls a reusable subprocess named \"{task['calledElement']}\".
        Assume subprocess is governed and separately managed.
        """
    elif task.get("subtype") == "embedded":
        prompt += """
        This is an embedded SubProcess. Generate a nested method or function that handles the internal logic inline.
        """

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [ ]:
# Generate full project code

In [9]:
def generate_code_from_bpmn(bpmn_path, platform_yaml_path, output_folder="generated_code"):
    tasks = parse_bpmn_tasks(bpmn_path)
    platform_spec = load_platform_spec(platform_yaml_path)
    os.makedirs(output_folder, exist_ok=True)

    ext = {
        "java": "java",
        "python": "py",
        "csharp": "cs"
    }.get(platform_spec['language'].lower(), "txt")

    for task in tasks:
        print(f"🔄 Generating code for: {task['name']} ({task['type']})")
        code = generate_task_code(task, platform_spec)

        folder = os.path.join(output_folder, "subprocesses") if task.get('subtype') == 'call-activity' else output_folder
        os.makedirs(folder, exist_ok=True)
        filename = os.path.join(folder, f"{task['name'].replace(' ', '_')}.{ext}")

        with open(filename, "w") as f:
            f.write(code)

        print(f"✅ Saved to {filename}")

In [ ]:
# Run the generator
# Replace with your file paths

In [11]:
bpmn_path = "ai.bpmn"  # Replace with your .bpmn file
platform_yaml_path = "platform.yaml"

generate_code_from_bpmn(bpmn_path, platform_yaml_path)

🔄 Generating code for: Collect Customer & Coverage Info (QuoteAgent) (task)
✅ Saved to generated_code/Collect_Customer_&_Coverage_Info_(QuoteAgent).java
🔄 Generating code for: Assess Risk (RiskAgent) (task)
✅ Saved to generated_code/Assess_Risk_(RiskAgent).java
🔄 Generating code for: Retry Risk Assessment (task)
✅ Saved to generated_code/Retry_Risk_Assessment.java
🔄 Generating code for: Calculate Quote (PricingAgent) (task)
✅ Saved to generated_code/Calculate_Quote_(PricingAgent).java
🔄 Generating code for: Underwrite Policy (UnderwritingAgent) (task)
✅ Saved to generated_code/Underwrite_Policy_(UnderwritingAgent).java
🔄 Generating code for: Human Underwriter Review (task)
✅ Saved to generated_code/Human_Underwriter_Review.java
🔄 Generating code for: Return Quote to Customer (QuoteAgent) (task)
✅ Saved to generated_code/Return_Quote_to_Customer_(QuoteAgent).java
🔄 Generating code for: Process Payment (PaymentAgent) (task)
✅ Saved to generated_code/Process_Payment_(PaymentAgent).java
🔄 